In [1]:
import mdtraj as md
import numpy as np
# import scipy.cluster.hierarchy
import os
# from tqdm import tqdm

In [2]:
phase = 'solvent' # do both solvent and complex (complex is more important)
proj = 13421 # will want to do both complex and solvent
RUNID = 0 # will want to loop over all run IDS

In [3]:
# first copy back pdb file, which contains the input topology needed
os.system(f'scp server@aws3.foldingathome.org:/home/server/server2/projects/{proj}/RUNS/RUN{RUNID}/*pdb .')

0

In [4]:
# load in the topology of the system
topology = md.load(f'hybrid_{phase}.pdb').topology

In [7]:
# now we want the final frame of the trajectory (for the forwards direction) we will want the starting frame also 
# (double check with John which frames correspond to which stages in the cycle)

# frame 0 is start ligand after EQUIL
# 0 -> 1 NEQ
# frame 1 is end ligand after NEQ
# frame 2 is end ligand after EQ
# 1 -> 0 NEQ
# frame 3 is start ligand after NEQ

#for i in {0..99}; do echo $i ; scp server@aws3.foldingathome.org:/home/server/server2/data/SVR314342810/PROJ13420/RUN0/CLONE0/results0/*xtc pos_$i.xtc ; done 
os.system(f'scp server@aws3.foldingathome.org:/home/server/server2/data/SVR314342810/PROJ{proj}/RUN{RUNID}/CLONE0/results0/*xtc pos_0.xtc')
traj = md.load_xtc(f'pos_0.xtc',topology,frame=2)

# for each clone, get the snapshot and append it to the trajectory
# will also want to do this over different GENS in future
n_clones = 4 # will be 100 
n_gens = 1 # will be 6 when everything is done
for gen in range(0,n_gens): 
    for clone in range(1,n_clones):
        print(clone)
        os.system(f'scp server@aws3.foldingathome.org:/home/server/server2/data/SVR314342810/PROJ{proj}/RUN{RUNID}/CLONE{clone}/results{gen}/*xtc pos_{clone}.xtc')
        new_frame = md.load_xtc(f'pos_{clone}.xtc',topology,frame=3)
        traj = traj.join(new_frame)
    

1
2
3


In [8]:
print(traj.n_frames) # this should match the number of (clones x gens)

4


In [9]:
# find the atom indices for the ligand (resname 'MOL')
mol_ids = []
for res in topology.residues:
    if res.name == 'MOL':
        for atom in res.atoms:
            mol_ids.append(atom.index)

In [10]:
# align the frames
traj.superpose(traj,atom_indices=mol_ids)
#only keep the ligand 
traj.atom_slice(mol_ids, inplace=True)
traj.save_pdb('testing.pdb')

In [ ]:
###
# now do the clustering
# find 'centroid' of each clusters
# additionally --- could 'weigh' configurations by work values, but I think we would want frame 1 for this? not sure